In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

## Check installation

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import matplotlib.pyplot as plt
import cv2

%matplotlib inline

## Download weights

In [ ]:
%bash

cd ../scripts/ && sh download_weights.sh

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = '/mmdetection/configs/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco.py'
# Setup a checkpoint file to load
checkpoint = '/mmdetection/weights/htc.pth'
# initialize the detector
#model = init_detector(config, checkpoint, device='cuda:1')

## Init Detector to inference from config and weights

In [1]:
main_config = 'configs/htc/htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco.py'
main_checkpoint = 'weights/htc.pth'

In [2]:
from mmcv import Config
from mmdet.apis import set_random_seed

cfg = Config.fromfile(main_config)

# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.classes = ('pig', )
cfg.data_root = '/mmdetection/data/datasets'
cfg.train_pipeline.insert(
    4,
    dict(
        type='CutOut',
        n_holes=(0, 4),
        cutout_ratio=[(0.02, 0.04), (0.04, 0.02), (0.03, 0.05), (0.02, 0.03)],
    )
)

cfg.data.samples_per_gpu = 3
cfg.data.workers_per_gpu = 4

cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = cfg.data_root
cfg.data.test.ann_file = 'annotations/test_all130.json'
cfg.data.test.img_prefix = 'test_img/'
cfg.data.test.classes = cfg.classes

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = cfg.data_root
cfg.data.train.ann_file = 'annotations/train_all290.json'
cfg.data.train.img_prefix = 'train_img/'
cfg.data.train.seg_prefix = 'semantic_train/'
cfg.data.train.classes = cfg.classes

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = cfg.data_root
cfg.data.val.ann_file = 'annotations/test_topview80.json'
cfg.data.val.img_prefix = 'test_topview_img'
cfg.data.val.classes = cfg.classes

# modify num classes of the model in box head
for i in range(len(cfg.model.roi_head.bbox_head)):
    cfg.model.roi_head.bbox_head[i].num_classes = 1
    
for i in range(len(cfg.model.roi_head.mask_head)):
    cfg.model.roi_head.mask_head[i].num_classes = 1
    
cfg.model.roi_head.semantic_head.num_classes = 2

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = main_checkpoint

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.002 / 8
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = ['bbox', 'segm']
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = [1, ]


# We can initialize the logger for training and have a look
# at the final config used for training
#print(f'Config:\n{cfg.pretty_text}')

In [3]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.models import build_detector


#model = build_detector(
#    cfg.model,
#    train_cfg=cfg.get('train_cfg'),
#    test_cfg=cfg.get('test_cfg'),
#)
#model.CLASSES = ('pig', )
#model.cfg = cfg

model = init_detector(cfg, main_checkpoint, device='cuda:1')

load checkpoint from local path: /mmdetection/notebooks/tutorial_exps/latest.pth


2021-11-19 19:42:06,757 - root - INFO - DeformConv2dPack backbone.layer2.0.conv2 is upgraded to version 2.
2021-11-19 19:42:06,788 - root - INFO - DeformConv2dPack backbone.layer2.1.conv2 is upgraded to version 2.
2021-11-19 19:42:06,808 - root - INFO - DeformConv2dPack backbone.layer2.2.conv2 is upgraded to version 2.
2021-11-19 19:42:06,828 - root - INFO - DeformConv2dPack backbone.layer2.3.conv2 is upgraded to version 2.
2021-11-19 19:42:06,857 - root - INFO - DeformConv2dPack backbone.layer3.0.conv2 is upgraded to version 2.
2021-11-19 19:42:06,889 - root - INFO - DeformConv2dPack backbone.layer3.1.conv2 is upgraded to version 2.
2021-11-19 19:42:06,916 - root - INFO - DeformConv2dPack backbone.layer3.2.conv2 is upgraded to version 2.
2021-11-19 19:42:06,958 - root - INFO - DeformConv2dPack backbone.layer3.3.conv2 is upgraded to version 2.
2021-11-19 19:42:06,986 - root - INFO - DeformConv2dPack backbone.layer3.4.conv2 is upgraded to version 2.
2021-11-19 19:42:07,030 - root - INFO

In [ ]:
img = '/mmdetection/data/datasets/test_frontview_img/287_B102_000210.png'
result = inference_detector(model, img)

# Let's plot the result
show_result_pyplot(model, img, result, score_thr=0.3)